# Main Telescope Slew simulation: Setup notebook

....

This notebook does slew simulations, and check all aos components (M1M3, M2, hexapods) behavior during the slew-and-track process

This is expected to work both for SUMMIT and NCSA

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import rubin_jupyter_utils.lab.notebook as nb
nb.utils.get_node()

In [ ]:
import os
import sys
import asyncio
import logging

import pandas as pd

from matplotlib import pyplot as plt

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel.mtcs import MTCS


In [ ]:
summit = 1 #use this for summit testing
# summit = 0 #use this for NCSA

## Check environment setup

The following cell will print some of the basic DDS configutions.

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])
print(os.environ.get("LSST_DDS_DOMAIN_ID", "Expected, not set."))

### Setup logging

Setup logging in debug mode and create a logger to use on the notebook.

In [ ]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [ ]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

# Starting communication resources

We start by creating a domain and later instantiate the MTCS class.
We will use the class to startup the components. 

In [ ]:
domain = salobj.Domain()

In [ ]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

# Starting components

From now on we will start the various components of the MTAOS.
You may wonder why are we not simply sending all CSCs to ENABLED state in one go, as we usually do on other systems.

The answer is that the MTCS components have some initilization dependencies that need to be observed for the components to be enabled properly.
We will describe these as we work our way the initialization steps.


## Starting MTPtg

We start by making sure the pointing component is alive, by waiting for a heartbeat.
Next we enable the component using `mtcs.set_state` method.

We select to start with the `MTPtg` mainly because, of all components of the `MTCS` it is the only pure-software components.
As such the `MTPtg` is pretty independent and can be brought to enabled in any condition.

It is also worth noticed that, as a pure-software component, the `MTPtg` does not have a simulation mode.

Furthermore, as you will notice below, we are not checking the software version of the `MTPtg`, mainly because the component is currently not sending this information.

In [ ]:
await mtcs.next_heartbeat("mtptg")

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtptg"])

## Starting MTMount

This is one case where the initialization order is important. 

The MTMount needs to be enabled before we enable the MTRotator.
The reason is that the MTRotator needs to know the position of the Camera Cable Wrap (CCW), which is provided by the MTMount, before it can be enable. 
If the MTRotator does not receive the position of the CCW, it will immediatelly activate the breaks and transition to FAULT state.

We start by verifying that the CSC is sending heartbeats.

In [ ]:
await mtcs.next_heartbeat("mtmount")

Now we can enable the CSC.

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtmount"])

### Perform some basic checks

The following are a couple of sanity checks we routinely perform when starting the MTMount.

We check if the CSC is running in simulation mode and then the version of the CSC.

Finally, we verify that the camera cable wrap following is enabled.

In [ ]:
mtmount_simulation_mode = await mtcs.get_simulation_mode(["mtmount"])

mode = mtmount_simulation_mode["mtmount"].mode
timestamp = pd.to_datetime(mtmount_simulation_mode["mtmount"].private_sndStamp, unit='s')

log.debug(
    f"MTMount simulation mode: {mode} @ {timestamp}"
)

In [ ]:
mtmount_software_versions = await mtcs.get_software_versions(["mtmount"])

csc_version = mtmount_software_versions["mtmount"].cscVersion
timestamp = pd.to_datetime(mtmount_software_versions["mtmount"].private_sndStamp, unit='s')

log.debug(
    f"MTMount software version: {csc_version} @ {timestamp}",
)

In [ ]:
mtmount_ccw_following = await mtcs.rem.mtmount.evt_cameraCableWrapFollowing.aget()

timestamp = pd.to_datetime(mtmount_ccw_following.private_sndStamp, unit='s')

if mtmount_ccw_following.enabled:
    log.debug(f"CCW following mode enabled: {mtmount_ccw_following.enabled} @ {timestamp}.")
else:
    await mtcs.set_state(salobj.State.DISABLED, ["mtmount"])
    raise RuntimeError(
        "CCW following mode not enabled. Usually this means that the MTMount could "
        "not see telemetry from the rotator when it was enabled. To correct this condition "
        "make sure the MTRotator telemetry is being published, then execute the procedure again. "
        "MTMount CSC will be left in DISABLED state."
        )


## Starting Rotator

In [ ]:
await mtcs.next_heartbeat("mtrotator")

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtrotator"])

### Perform some basic checks

The following is a few sanity checks we routinely perform to verify the system integrity at this stage.

In [ ]:
mtrotator_simulation_mode = await mtcs.get_simulation_mode(["mtrotator"])

mode = mtrotator_simulation_mode["mtrotator"].mode
timestamp = pd.to_datetime(mtrotator_simulation_mode["mtrotator"].private_sndStamp, unit='s')

log.debug(
    f"MTRotator simulation mode: {mode} @ {timestamp}"
)

In [ ]:
mtrotator_software_versions = await mtcs.get_software_versions(["mtrotator"])

csc_version = mtrotator_software_versions["mtrotator"].cscVersion
timestamp = pd.to_datetime(mtrotator_software_versions["mtrotator"].private_sndStamp, unit='s')

log.debug(
    f"MTRotator software version: {csc_version} @ {timestamp}",
)

In [ ]:
elevation = await mtcs.rem.mtmount.tel_elevation.next(flush=True, timeout=5)
azimuth = await mtcs.rem.mtmount.tel_azimuth.next(flush=True, timeout=5)
ccw = await mtcs.rem.mtmount.tel_cameraCableWrap.next(flush=True, timeout=5)
rotator = await mtcs.rem.mtrotator.tel_rotation.next(flush=True, timeout=5)

log.info(f"mount elevation Angle = {elevation.actualPosition}")
log.info(f"mount azimuth angle = {azimuth.actualPosition}")
log.info(f"CCW angle = {ccw.actualPosition}. Needs to be within 2.2 deg of rotator angle ")
log.info(f"rot angle = {rotator.actualPosition} diff = {rotator.actualPosition - ccw.actualPosition}")

### CCW telemetry too old

This warning message may appear in the `MTRotator` in a couple different conditions.

The most common occurence is when the `MTMount` component is not publishing the CCW telemetry.
This should be rectified by enabling the CSC, as we've done on the section above, and is one of the reasons we enable `MTMount` before the `MTRotator`.

The less common but more critical condition is when the clock on the `MTMount` controller is out of sync with the observatory clock server.
In this case, the `timestamp` attribute, used by the `MTRotator` to determine the relevant time for the published telemetry, will be out of sync and we won't be able to operate the system.

You can use the cell below to determine whether this is the case or not.
If so, you need to contact IT or someone with knowledge about the `MTMount` low level controller to fix the time synchronization issue.



In [ ]:
ccw = await mtcs.rem.mtmount.tel_cameraCableWrap.next(flush=True, timeout=5)
rotator = await mtcs.rem.mtrotator.tel_rotation.next(flush=True, timeout=5)

ccw_snd_stamp = pd.to_datetime(ccw.private_sndStamp, unit='s')
ccw_timestamp = pd.to_datetime(ccw.timestamp, unit='s')
ccw_actual_position = ccw.actualPosition

rotator_snd_stamp = pd.to_datetime(rotator.private_sndStamp, unit='s')
rotator_timestamp = pd.to_datetime(rotator.timestamp, unit='s')
rotator_actual_position = rotator.actualPosition

log.info(
    f"CCW:: snd_stamp={ccw_snd_stamp} timestamp={ccw_timestamp} actual position={ccw_actual_position}"
    )
log.info(
    f"Rotator:: snd_stamp={rotator_snd_stamp} timestamp={rotator_timestamp} actual position={rotator_actual_position}"
    )

ccw_telemetry_maximum_age = pd.to_timedelta(1.0, unit='s')

if abs(ccw_snd_stamp - ccw_timestamp) > ccw_telemetry_maximum_age:
    log.warning(
        f"CCW timestamp out of sync by {abs(ccw_snd_stamp - ccw_timestamp)}s. "
        "System may not work. Check clock synchronization in MTMount low level controller."
        )

### Clearing error in MTRotator

If the MTRotator is in FAULT state, you need to send the `clearError` command before transitioning it back to `ENABLED`.

This is a particularity of the `MTRotator` (and `MTHexapod`) that violates our state machine.

In [ ]:
if False:
    await mtcs.mtrotator.cmd_clearError.set_start()

## Checkpoint

At this point the system is ready for exercicing slew activities, without involving the optical components.

## Starting M1M3 (Mount telemetry mode)

If running the test on level 3 and if M1M3 is configured to listen for the mount telemetry, we firt need to make sure the `MTMount` is pointing to zenith.

The reason is that `M1M3` is in a fixed position and, when we try to enabled/raise it, the will check the inclinometer data against the elevation data. If they differ by more than a couple degrees the process will fail.

Once M1M3 is mounted on the telescope and we are operating the actual mount, instead of in simulation mode, this will not be necessary.

In [ ]:
await mtcs.rem.mtmount.cmd_moveToTarget.set_start(azimuth=0, elevation=90)

In [ ]:
await mtcs.next_heartbeat("mtm1m3")

In [ ]:
await mtcs.set_state(
    state=salobj.State.ENABLED,
    settings=dict(mtm1m3="Default"),
    components=["mtm1m3"]
    )

### Raise m1m3

Now that m1m3 is enabled we can raise it.

The following has a trick to allow raising the m1m3 in the background and give control back to the notebook. 
If, in middle of the process, you need to abort the operation you can still do it from the notebooks.

Once you execute the cell bellow you will notice that the log messages will appear below the cell, but you can also see that the cell will be masked as "finished executing".
That means, instead of seeing an `*` you will see the number of the cell.
This is because the operation is running in the background and we have control over the notebook to execute additional cells.

In [ ]:
task_raise_m1m3 = asyncio.create_task(mtcs.raise_m1m3())

The next cell contain a command to abort the raise operation initiated in the background on the cell above.
Note that the command to execute the abort operation is encapsulated by an `if False`.
This is to prevent the command from executing if the notebook is being executed by papermill or by accident.

If you need to abort the operation change the if statement to `if True`.

In [ ]:
if False:
    await mtcs.abort_raise_m1m3()


The next cell will wait for the raise_m1m3 command to finish executing. 
This is to make sure a batch processing of the notebook won't proceed until the raise operation is completed.

In [ ]:
await task_raise_m1m3

In [ ]:
await mtcs.enable_m1m3_balance_system()

In [ ]:
await mtcs.reset_m1m3_forces()

In [ ]:
# Move this to a shutdown notebook...
# await lowerM1M3(m1m3)

## Starting M2

In [ ]:
await mtcs.next_heartbeat("mtm2")

<div class="alert alert-block alert-info">
Remember to reset interlocks.
</div>


M2 has an issue that it returns the state transition commands before it is actually finishing doing the state transition.
This causes the subsequent transitions to fail.
To work around it we will do them one at a time, adding a sleep between each of them to allow the CSC to finish the state transition.

These workarounds should be removed once the CSC is fixed.

In [ ]:
await mtcs.set_state(
    state=salobj.State.STANDBY,
    components=["mtm2"]
    )

In [ ]:
mtm2_state_transition_sleep_time = 5.

In [ ]:
await asyncio.sleep(mtm2_state_transition_sleep_time)

In [ ]:
await mtcs.set_state(
    state=salobj.State.DISABLED,
    components=["mtm2"]
    )

In [ ]:
await asyncio.sleep(mtm2_state_transition_sleep_time)

In [ ]:
await mtcs.set_state(
    state=salobj.State.ENABLED,
    components=["mtm2"]
    )

In [ ]:
if False:
    await mtcs.rem.mtm2.cmd_clearErrors.set_start(timeout=15.)

### Prepare M2 for operation

Switch on m2 force balance system and reset m2 forces.

In [ ]:
await mtcs.enable_m2_balance_system()

In [ ]:
await mtcs.reset_m2_forces()

## Starting Camera Hexapod

In [ ]:
await mtcs.next_heartbeat("mthexapod_1")

The command bellow to enable the Camera Hexapod should work, in general. 
Nevertheless, we found an issue with the interaction between the low level controller and the CSC that was causing it to fail from time to time.

The error report can be found in [DM-31111](https://jira.lsstcorp.org/browse/DM-31111).

Until this ticket is worked on you may encounter failures when executing the cell below.
You can continue by running the cell again.

In [ ]:
await mtcs.set_state(
    state=salobj.State.ENABLED,
    settings=dict(mthexapod_1="default"),
    components=["mthexapod_1"]
    )

In [ ]:
mthexapod_1_simulation_mode = await mtcs.get_simulation_mode(["mthexapod_1"])

mode = mthexapod_1_simulation_mode["mthexapod_1"].mode
timestamp = pd.to_datetime(mthexapod_1_simulation_mode["mthexapod_1"].private_sndStamp, unit='s')

log.debug(
    f"Camera Hexapod simulation mode: {mode} @ {timestamp}"
)

In [ ]:
mthexapod_1_software_versions = await mtcs.get_software_versions(["mthexapod_1"])

csc_version = mthexapod_1_software_versions["mthexapod_1"].cscVersion
timestamp = pd.to_datetime(mthexapod_1_software_versions["mthexapod_1"].private_sndStamp, unit='s')

log.debug(
    f"Camera Hexapod software version: {csc_version} @ {timestamp}",
)

In [ ]:
if False:
    await mtcs.rem.mthexapod_1.cmd_clearError.set_start()

In [ ]:
await mtcs.enable_compensation_mode(component="mthexapod_1")

In [ ]:
await mtcs.reset_camera_hexapod_position()

## Starting M2 Hexapod

In [ ]:
await mtcs.next_heartbeat("mthexapod_2")

We have been mostly running the M2 Hexapod in simulation mode, because the actual hardware is mounted on the telescope.
This means the M2 Hexapod is not affected by the issue we reported above for the Camera Hexapod.

In [ ]:
await mtcs.set_state(
    state=salobj.State.ENABLED,
    settings=dict(mthexapod_2="default"),
    components=["mthexapod_2"]
    )

In [ ]:
mthexapod_2_simulation_mode = await mtcs.get_simulation_mode(["mthexapod_2"])

mode = mthexapod_2_simulation_mode["mthexapod_2"].mode
timestamp = pd.to_datetime(mthexapod_2_simulation_mode["mthexapod_2"].private_sndStamp, unit='s')

log.debug(
    f"M2 Hexapod simulation mode: {mode} @ {timestamp}"
)

In [ ]:
mthexapod_2_software_versions = await mtcs.get_software_versions(["mthexapod_2"])

csc_version = mthexapod_2_software_versions["mthexapod_2"].cscVersion
timestamp = pd.to_datetime(mthexapod_2_software_versions["mthexapod_2"].private_sndStamp, unit='s')

log.debug(
    f"M2 Hexapod software version: {csc_version} @ {timestamp}",
)

In [ ]:
await mtcs.enable_compensation_mode(component="mthexapod_2")

In [ ]:
await mtcs.reset_camera_hexapod_position()

# Closing MTCS and Domain


In [ ]:
await mtcs.close()

In [ ]:
await domain.close()